In [ ]:
!pip install python-twitter python-dotenv

In [1]:
import os
import json
import time
import urllib
import twitter
import itertools
import datetime
import pandas as pd

from dotenv import load_dotenv
load_dotenv()

t = twitter.Api(consumer_key=os.getenv('CONSUMER_KEY'),
                consumer_secret=os.getenv('CONSUMER_SECRET'),
                access_token_key=os.getenv('ACCESS_TOKEN'),
                access_token_secret=os.getenv('ACCESS_SECRET'),
                tweet_mode='extended')

def remove_starting_mentions(tw):
    text = tw.full_text.strip()
    for user in set([u.screen_name for u in tw.user_mentions]):
        text = text.replace(f'@{user}', '', 1)
    text = text.strip()
    return text

def parse_date(dt):
    return datetime.datetime.strptime(dt, '%a %b %d %H:%M:%S %z %Y')

In [30]:
tweets = {}

# https://twitter.com/ohbmequinox/status/1240818088376823809?s=21

first_tweet = 1240818088376823809
params      = {
    'include_rts': False,
    'count': 5000,
    'since_id': first_tweet-1,
}

fetching= True
while fetching:
    fetching = t.GetUserTimeline(screen_name='ohbmequinox', **params)
    tweets.update({tt.id: tt for tt in fetching})
    if not fetching or first_tweet in [tt.id for tt in fetching]:
        break
    params = {'max_id': fetching[-1].id}
    
# https://twitter.com/AskDrJeg/status/1240855036873211904
wrong_status = t.GetStatus(1240855036873211904)
wrong_status.full_text = wrong_status.full_text.replace('@OHBMequinoX ', '', 1)
wrong_status.user_mentions = wrong_status.user_mentions[1:]
wrong_status.user = list(tweets.values())[0].user
tweets.update({wrong_status.id: wrong_status})
del tweets[1240854713781764101]

# https://twitter.com/OHBMequinoX/status/1240823872976379904
del tweets[1240823872976379904]

In [41]:
originals = {twid: tw for twid, tw in tweets.items() if tw.retweeted_status is None}

timeline = {
    twid: {
        'date': parse_date(tw.created_at),
        'text': tw.full_text.strip(),
        'ids': [twid],
        'mentions': [u.id for u in tw.user_mentions],
    }
    for twid, tw in originals.items()
    if tw.in_reply_to_status_id is None
}

print({k: t for k, t in timeline.items() if 'honour' in t['text']})

originals_replies = {
    twid: tw for twid, tw in originals.items()
    if tw.in_reply_to_status_id and tw.in_reply_to_screen_name.lower() in ['ohbmequinox']
}

{1240912684163334145: {'date': datetime.datetime(2020, 3, 20, 8, 6, 53, tzinfo=datetime.timezone.utc), 'text': 'It’s been an honour to bring you #OHBMx from Aus. @LeonieBorne\n @KoussisNikitas\n @AskDrJeg\n @DrBreaky\n @MegaEJ_Campbell\n signing off. \nThoughts are with our EU colleagues &amp; their families in this challenging time. \nPls take a break before the EU Hub takes the helm.', 'ids': [1240912684163334145], 'mentions': [1228144941315936257, 1033212944832593920, 1135512246438678528, 3305807792, 1886474232]}}


In [42]:
reply_to = {twid: twid for twid in timeline}

while originals_replies:

    for twid, tw in list(originals_replies.items()):
        
        twid_reply = tw.in_reply_to_status_id

        if twid_reply not in timeline and twid_reply not in reply_to:
            continue
            
        if twid_reply in reply_to:
            twid_reply = reply_to[twid_reply]
            
        reply_to[twid] = twid_reply

        original_text = timeline[twid_reply]['text']
        text = remove_starting_mentions(tw)

        for ellipsis in ['...', '…']:
            if text.startswith(ellipsis):
                text = text.replace(ellipsis, '', 1)
            if original_text.endswith(ellipsis):
                original_text = ellipsis.join(original_text.split(ellipsis)[:-1])

        timeline[twid_reply]['text'] = f'{original_text} {text}'
        
        if twid_reply == 1240888314137202689:
            timeline[twid_reply]['text'] = timeline[twid_reply]['text'].replace('1/2', '').replace('2/2', '')
        
        timeline[twid_reply]['ids'] += [twid]
        timeline[twid_reply]['mentions'] += [u.id for u in tw.user_mentions]
        
        del originals_replies[twid]

In [43]:
timeline[1240912684163334145]

{'date': datetime.datetime(2020, 3, 20, 8, 6, 53, tzinfo=datetime.timezone.utc),
 'text': 'It’s been an honour to bring you #OHBMx from Aus. @LeonieBorne\n @KoussisNikitas\n @AskDrJeg\n @DrBreaky\n @MegaEJ_Campbell\n signing off. \nThoughts are with our EU colleagues &amp; their families in this challenging time. \nPls take a break before the EU Hub takes the helm.',
 'ids': [1240912684163334145],
 'mentions': [1228144941315936257,
  1033212944832593920,
  1135512246438678528,
  3305807792,
  1886474232]}

In [44]:
df = pd.DataFrame([
    {
        'id': twid,
        'date': tw['date'],
        'text': tw['text'],
        'session': tw['text'].split('\n')[0],
    }
    for twid, tw in timeline.items()
    if tw['text'].startswith('#OHBMx-')
]).sort_values(by='date')

In [45]:
df.to_json('presentations.json', orient='records')

In [46]:
threads = df[(df.session.str.contains('#keynote') | df.session.str.contains('#talk'))]

In [8]:
def get_replies(tweets, mentions):
    user = tweets[0].user.screen_name
    tweet_ids = [tweet.id for tweet in tweets]
    
    max_id = None
    while True:
        try:
            replies = t.GetSearch(
                term=f'(to:@{user})',
                since_id=min(tweet_ids),
                max_id=max_id,
                count=100,
                result_type='recent'
            )
        except twitter.error.TwitterError as e:
            print("Waiting")
            time.sleep(60)
            continue
            
        replied_tweets = []
        for reply in replies:
            
            max_id = reply.id
            
            if reply.in_reply_to_status_id not in tweet_ids:
                continue
                
            if reply.user.id not in mentions[reply.in_reply_to_status_id]:
                continue
                
            mentions.update({
                reply.id: mentions[reply.in_reply_to_status_id]
            })
                
            yield reply
            yield from get_replies([reply], mentions)
            
        if len(replies) != 100:
            break

In [9]:
all_tweets_from_threads = []
mentions_for_tweets_from_threads = {}
for twid in threads.id.tolist():
    all_tweets_from_threads += timeline[twid]['ids']
    for subtwid in timeline[twid]['ids']:
        mentions_for_tweets_from_threads[subtwid] = timeline[twid]['mentions']
        
replies = list(get_replies([tweets[twid] for twid in all_tweets_from_threads], mentions_for_tweets_from_threads))

Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting


In [62]:
def unthread(tweets, original_tweets, replies):
    thread_replies = [
        tw for tw in replies
        if tw.in_reply_to_status_id in original_tweets and tw.id not in original_tweets
    ]
    if thread_replies:
        yield from thread_replies
        yield from unthread(tweets, [r.id for r in thread_replies], replies)

for twid in timeline:
    if twid not in all_tweets_from_threads:
        continue
        
    timeline[twid]['thread'] = []
    for rr in sorted(unthread(tweets, timeline[twid]['ids'], replies), key=lambda tw: (parse_date(tw.created_at))):
        timeline[twid]['thread'] += [{
            'id': str(rr.id),
            'user': rr.user.screen_name,
            'date': parse_date(rr.created_at),
            'text': remove_starting_mentions(rr),
            'media': rr.AsDict().get('media', []),
        }]

In [63]:
data = { **{twid: tw for twid, tw in tweets.items()}, **{r.id: r for r in replies} }

In [64]:
import json
with open('data.json', 'w') as outfile:
    json.dump(
        {'tweets': sorted([d.AsDict() for d in data.values()], key=lambda t: parse_date(t['created_at']))},
        outfile
    )

In [65]:
import json

def dtconverter(o):
    if isinstance(o, datetime.datetime):
        return o.__str__()
 
with open('timeline.json', 'w') as outfile:
    json.dump(
        {
            'timeline': sorted([
                { **t, 'ids': [str(twid) for twid in t['ids']]}
                for t in timeline.values()
            ], key=lambda t: t['date'])
        },
        outfile,
        default=dtconverter
    )